## 전처리 미포함

In [5]:
import pandas as pd

review_jungrang_ori = pd.read_csv('C:/Users/user/Desktop/saltlux_project/preprocess/JungrangTotal.csv', index_col=0)
review_jungrang_ori = review_jungrang_ori[['name', 'nickname', 'content']].reset_index(drop=True)

In [6]:
review_jungrang_ori['content'] = review_jungrang_ori['content'].str.replace('\n', ' ')
review_jungrang_ori = review_jungrang_ori.reset_index(drop=True)
review_jungrang_ori

,name,nickname,content
0,365mc모인이비인후과의원,Wiseburge David,사가정 모인 이비인후과 항상 만원이다
1,365mc모인이비인후과의원,Dk9,좋아요
2,365mc모인이비인후과의원,귀한집딸z,좋아요
3,365mc모인이비인후과의원,꽃길걷는중임,항상 친절한 진료 감사합니다~
4,365mc모인이비인후과의원,막둥,ㅇㅇ
...,...,...,...
5970,효치과의원,fpvocalist,굳
5971,효치과의원,소보루39,좋아요
5972,효치과의원,1vvovv1,친절하십니다 과잉진료 없음
5973,효치과의원,버럭아저씨,친절해요..


In [7]:
# gpu 사용
import torch
device = torch.device("cuda:0")

In [2]:
from kobert.pytorch_kobert import get_kobert_model
from transformers import BertModel
from kobert_hf.kobert_tokenizer.kobert_tokenizer import KoBERTTokenizer
from torch.nn.functional import softmax

import torch

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# KoBERT 모델 및 토크나이저 불러오기
model_name = 'skt/kobert-base-v1'
tokenizer = KoBERTTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 결과를 저장할 리스트 초기화
results = []

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        logits = model(**tokens).logits

    probs = softmax(logits, dim=1).squeeze().tolist()
    result_str = f"긍정 확률: {probs[1]:.2%}, 부정 확률: {probs[0]:.2%}"
    results.append({'Text': text, 'Result': result_str})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('kobert_results_ori_prob.csv', index=False)


ModuleNotFoundError: No module named 'gluonnlp'

In [ ]:
pip install gluonnlp==0.9.1
pip install mxnet==1.6.0
pip install boto3==1.15
pip install onnxruntime 

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# 모델과 토크나이저를 직접 다운로드
model_name = 'monologg/kobert'
model_path = 'monologg_kobert'
tokenizer_path = 'monologg_kobert/tokenizer'

tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# 결과를 저장할 리스트 초기화
results = []

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        logits = model(**tokens).logits

    probs = softmax(logits, dim=1).squeeze().tolist()
    result_str = f"긍정 확률: {probs[1]:.2%}, 부정 확률: {probs[0]:.2%}"
    results.append({'Text': text, 'Result': result_str})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('kobert_results_ori_prob.csv', index=False)

In [11]:
import torch
from transformers import BertModel, DistilBertModel
from kobert_transformers import get_tokenizer

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# KoBERT 모델 및 토크나이저 불러오기
model_name = 'skt/kobert-base-v1'
model = BertModel.from_pretrained(model_name)
tokenizer = get_tokenizer()

# 결과를 저장할 리스트 초기화
results = []

# 임계값 설정
threshold = 0.5

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        # 'last_hidden_state'를 사용하여 모델 출력 얻기
        outputs = model(**tokens)
    # 'last_hidden_state'에서 [CLS] 토큰에 해당하는 벡터 가져오기
    cls_token_embedding = outputs.last_hidden_state[:, 0, :]

    # 확률 계산 방법 적용 -> softmax
    probs = softmax(cls_token_embedding, dim=1).squeeze().tolist()
    result_str = f"긍정 확률: {probs[1]:.2%}, 부정 확률: {probs[0]:.2%}"
    results.append({'Text': text, 'Result': result_str})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('kobert_results_ori_prob.csv', index=False)

In [14]:
import torch
import torch.nn as nn
from transformers import BertModel
from kobert_transformers import get_tokenizer
from torch.nn.functional import softmax
import pandas as pd

# 'content' 컬럼의 데이터를 리스트로 추출
texts = review_jungrang_ori['content'].tolist()

# KoBERT 모델 및 토크나이저 불러오기
model_name = 'skt/kobert-base-v1'
model = BertModel.from_pretrained(model_name)
tokenizer = get_tokenizer()

# 추가 분류 레이어 정의
classifier = nn.Linear(768, 2)  # KoBERT의 hidden size는 768

# 결과를 저장할 리스트 초기화
results = []

# 임계값 설정
threshold = 0.5

# 각 텍스트에 대해 확률 계산 및 결과 저장
for text in texts:
    tokens = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        # 'last_hidden_state'를 사용하여 모델 출력 얻기
        outputs = model(**tokens)
    
    # 'last_hidden_state'에서 [CLS] 토큰에 해당하는 벡터 가져오기
    cls_token_embedding = outputs.last_hidden_state[:, 0, :]
    
    # 추가 분류 레이어를 통과시켜 로짓 계산
    logits = classifier(cls_token_embedding)
    
    # 확률로 변환
    probs = softmax(logits, dim=1).squeeze().tolist()
    
    # 임계값을 기준으로 긍정과 부정을 나눔
    sentiment = "긍정" if probs[1] >= threshold else "부정"
    results.append({'Text': text, 'Result': sentiment})

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 결과를 CSV 파일로 저장
results_df.to_csv('kobert_results_ori_sentiment.csv', index=False)
